<a href="https://colab.research.google.com/github/dnth/yolov5-deepsparse-blogpost/blob/master/notebooks/deepsparse_blogpost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!git clone https://github.com/dnth/yolov5-deepsparse-blogpost

DeepSparse only works with PyTorch 1.9.0 for now. Let's install that version.

In [ ]:
!pip install torch==1.9.0 torchvision==0.10.0 --extra-index-url https://download.pytorch.org/whl/cu111

In [ ]:
!cd yolov5-deepsparse-blogpost && pip install -r req.txt

# Training YOLOv5-S Baseline

Let's train the baseline YOLOv5 model with no optimization.

In [ ]:
# Uncomment to train model
# !cd yolov5-deepsparse-blogpost/yolov5-train/ && python train.py \
#               --cfg ./models_v5.0/yolov5s.yaml \
#               --data pistols.yaml --hyp data/hyps/hyp.scratch.yaml \
#               --weights yolov5s.pt --img 416 --batch-size 64 --epochs 240 

# Training YOLOv5-S (Pruned)

Now let's trained a YOLOv5-S and prune it.

In [ ]:
# Uncomment to train model
# !cd yolov5-deepsparse-blogpost/yolov5-train/ && python train.py --cfg ./models_v5.0/yolov5s.yaml \
#               --recipe ../recipes/yolov5s.pruned.md \
#               --data pistols.yaml --hyp data/hyps/hyp.scratch.yaml \
#               --weights yolov5s.pt --img 416 --batch-size 64

# Training YOLOv5-S (Pruned + Quantized)

Finally let's train a YOLOv5 with pruning and quantization.

In [ ]:
# Uncomment to train model
# !cd yolov5-deepsparse-blogpost/yolov5-train/ && python train.py --cfg ./models_v5.0/yolov5s.yaml \
#               --recipe ../recipes/yolov5.transfer_learn_pruned_quantized.md \
#               --data pistols.yaml --hyp data/hyps/hyp.scratch.yaml \
#               --weights yolov5s.pt --img 416 --batch-size 64

# Training YOLOv5-Nano (Pruned + Quantized)

In [ ]:
# Uncomment to train model
# !cd yolov5-deepsparse-blogpost/yolov5-train/ && python train.py --cfg ./models_v5.0/yolov5n.yaml \
#                --recipe ../recipes/yolov5.transfer_learn_pruned_quantized.md \
#                --data pistols.yaml --hyp data/hyps/hyp.scratch.yaml \
#                --weights yolov5n.pt --img 416 --batch-size 64

# Export to ONNX

Let's export the pruned+quantized model to ONNX.

In [ ]:
!cd yolov5-deepsparse-blogpost/yolov5-train/ && python export.py --weights yolov5-deepsparse/yolov5s-sgd-pruned-quantized-transfer/weights/best.pt --include onnx --imgsz 416 --dynamic --simplify

export: data=data/coco128.yaml, weights=['yolov5-deepsparse/yolov5s-sgd-pruned-quantized-transfer/weights/best.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=True, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, remove_grid=False, include=['onnx']
YOLOv5 🚀 e3e5ba3 torch 1.9.0+cu111 CPU


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               

# Run Inference

Run inference with the YOLOv5-Nano

In [ ]:
!cd yolov5-deepsparse-blogpost/yolov5-train/ && python annotate.py yolov5-deepsparse/yolov5n-sgd-pruned-quantized-hardswish/weights/best.onnx --source data/pexels-cottonbro-8717592.mp4 --engine deepsparse --device cpu --conf-thres 0.7 --image-shape 416 416 --quantized-input

# Resize the video to 640x480 to be displayed in Colab

In [ ]:
!ffmpeg -i yolov5-deepsparse-blogpost/yolov5-train/annotation_results/deepsparse-annotations/results.mp4 -vf scale=640:480 yolov5-deepsparse-blogpost/yolov5-train/annotation_results/deepsparse-annotations/output.mp4

# Display video on Colab

With 2 CPU cores on Colab we get about 8 FPS inference.

In [ ]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 640):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video("yolov5-deepsparse-blogpost/yolov5-train/annotation_results/deepsparse-annotations/output.mp4")

# CPU Information for Colab

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
# Delete all annotation results
# !cd yolov5-deepsparse-blogpost/yolov5-train/ && rm -rf annotation_results/